In [3]:
import pandas as pd
import duckdb as ddb
import json
import schedule
import time 

In [4]:
def update():
    with open('response.json') as f:
        data = json.load(f)
    location_df = pd.DataFrame([data['location']])
    weather_df = pd.DataFrame([data['current']])
    weather_df
    flattened_data = {**data["location"], **data["current"]}

    # Convert to a DataFrame
    df = pd.DataFrame([flattened_data])
    df = df.drop(columns='condition')
    df.columns.value_counts().sum()
    conn = ddb.connect('../weatherdata')
    conn.execute("CREATE SCHEMA IF NOT EXISTS raw")
    conn.execute("""
             CREATE TABLE IF NOT EXISTS raw.DAILY_WEATHER_DATA (
                    name VARCHAR(100),
                    region VARCHAR(100),
                    country VARCHAR(100),
                    lat FLOAT,
                    lon FLOAT,
                    tz_id VARCHAR(100),
                    localtime_epoch BIGINT,
                    localtime DATETIME,
                    last_updated_epoch BIGINT,
                    last_updated DATETIME,
                    temp_c FLOAT,
                    temp_f FLOAT,
                    is_day TINYINT,
                    wind_mph FLOAT,
                    wind_kph FLOAT,
                    wind_degree SMALLINT,
                    wind_dir VARCHAR(10),
                    pressure_mb FLOAT,
                    pressure_in FLOAT,
                    precip_mm FLOAT,
                    precip_in FLOAT,
                    humidity TINYINT,
                    cloud TINYINT,
                    feelslike_c FLOAT,
                    feelslike_f FLOAT,
                    windchill_c FLOAT,
                    windchill_f FLOAT,
                    heatindex_c FLOAT,
                    heatindex_f FLOAT,
                    dewpoint_c FLOAT,
                    dewpoint_f FLOAT,
                    vis_km FLOAT,
                    vis_miles FLOAT,
                    uv FLOAT,
                    gust_mph FLOAT,
                    gust_kph FLOAT,
                    PRIMARY KEY (localtime_epoch)
                );

             """)
    conn.execute("INSERT INTO raw.DAILY_WEATHER_DATA SELECT * FROM df")
    conn.close()

    

In [5]:
# with open('response.json') as f:
#     data = json.load(f)

In [6]:
# location_df = pd.DataFrame([data['location']])
# weather_df = pd.DataFrame([data['current']])
# weather_df

In [7]:
# flattened_data = {**data["location"], **data["current"]}

# # Convert to a DataFrame
# df = pd.DataFrame([flattened_data])
# df

In [8]:
# df.columns

In [9]:
# df = df.drop(columns='condition')

In [10]:
# df.columns.value_counts().sum()

In [11]:
# conn = ddb.connect('../weatherdata')


In [12]:
# conn.execute("CREATE SCHEMA IF NOT EXISTS raw")

In [13]:
# conn.execute("""
#              CREATE TABLE IF NOT EXISTS raw.DAILY_WEATHER_DATA (
#                     name VARCHAR(100),
#                     region VARCHAR(100),
#                     country VARCHAR(100),
#                     lat FLOAT,
#                     lon FLOAT,
#                     tz_id VARCHAR(100),
#                     localtime_epoch BIGINT,
#                     localtime DATETIME,
#                     last_updated_epoch BIGINT,
#                     last_updated DATETIME,
#                     temp_c FLOAT,
#                     temp_f FLOAT,
#                     is_day TINYINT,
#                     wind_mph FLOAT,
#                     wind_kph FLOAT,
#                     wind_degree SMALLINT,
#                     wind_dir VARCHAR(10),
#                     pressure_mb FLOAT,
#                     pressure_in FLOAT,
#                     precip_mm FLOAT,
#                     precip_in FLOAT,
#                     humidity TINYINT,
#                     cloud TINYINT,
#                     feelslike_c FLOAT,
#                     feelslike_f FLOAT,
#                     windchill_c FLOAT,
#                     windchill_f FLOAT,
#                     heatindex_c FLOAT,
#                     heatindex_f FLOAT,
#                     dewpoint_c FLOAT,
#                     dewpoint_f FLOAT,
#                     vis_km FLOAT,
#                     vis_miles FLOAT,
#                     uv FLOAT,
#                     gust_mph FLOAT,
#                     gust_kph FLOAT,
#                     PRIMARY KEY (localtime_epoch)
#                 );

#              """)

In [14]:
# conn.execute("INSERT INTO raw.DAILY_WEATHER_DATA SELECT * FROM df")

In [15]:
# conn.close()

In [17]:
schedule.every(40).seconds.do(update)

while True:
    schedule.run_pending()
    time.sleep(1)
#update()

ConstraintException: Constraint Error: Duplicate key "localtime_epoch: 1733288461" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (https://duckdb.org/docs/sql/indexes).